### **Eksik Değer Problemini Çözme**

In [2]:
##################### GEÇMİŞ TANIMLAMALAR ##############################

# Kütüphaneler
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
# pip install missingno
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler


# Pandas ayarları
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)


# Veri setleri
def load_application_train():
    data = pd.read_csv('../01-outliers/application_train.csv')
    return data

def load():
    data = pd.read_csv('../01-outliers/titanic.csv')
    return data


# Fonksiyonlar
# Eksik değer içeren değişkenleri dönen fonksiyon:
def missing_values_table(dataframe, na_name=False):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df,end='\n')
    if na_name:
        return na_columns

In [3]:
df = load()

In [4]:
missing_values_table(df)

          n_miss  ratio
Cabin        687 77.100
Age          177 19.870
Embarked       2  0.220


> 1. Çözüm: Hızlıca Silmek

In [5]:
df.dropna().shape
# 891 gözlemden 183 kaldı. Bu nedenle silmek pek mantıklı değil.

(183, 12)

> 2. Çözüm: Basit Atama Yöntemleri ile Doldurmak

In [6]:
# Ortalama veya medyan değerleriyle eksik değerleri doldurmak:

# Age değişkenine ait boş değerleri ortalama değer ile doldurunca eksik kalan değer sayısı: (0 olmalı)
df["Age"].fillna(df["Age"].mean()).isnull().sum()

0

In [7]:
# Age değişkenine ait boş değerleri medyan değer ile doldurunca eksik kalan değer sayısı: (0 olmalı)
df["Age"].fillna(df["Age"].median()).isnull().sum()

0

In [8]:
# Age değişkenine ait boş değerleri sabit bir değer olan 0 ile doldurunca eksik kalan değer sayısı: (0 olmalı)
df["Age"].fillna(0).isnull().sum()

0

In [9]:
# Bu işlemi veri setindeki bütün değişkenler için yapmak istersek
# df.apply(lambda x: x.fillna(x.mean()), axis=0) # Böyle yaparsak hata alırız çünkü sayısal hariç kategorik 
# değerlerin de ortalamasını almaya çalışır ve hata verir.

# axis = 0 vermemizin sebebi gözlemlerin ortalamasını almak istediğimiz içindir. Normalde axis = 1 olarak yaptığımız zaman sütunları ayırırız.

In [10]:
# Bu sorunu çözmek için fillna() fonksiyonunu sadece sayısal değişkenlere uygulamalıyız.
df.apply(lambda x: x.fillna(x.mean()) if x.dtype != "O" else x, axis=0).head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


In [11]:
dff = df.apply(lambda x: x.fillna(x.mean()) if x.dtype != "O" else x, axis=0)

In [12]:
dff.isnull().sum().sort_values(ascending=False)

Cabin          687
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

In [13]:
# Kategorik değişkenlerin eksik değerlerini doldurmanın en etkili yolu modunu (en çok tekrar eden değeri) ile değişkendeki eksik değerleri doldurmaktır.
df['Embarked'].fillna(df['Embarked'].mode()[0]).isnull().sum()

0

In [14]:
# Veri setindeki tüm kategorik değişkenlere mode() fonksiyonu ile eksik değelerini dolduralım.
df.apply(lambda x: x.fillna(x.mode()[0]) if ((x.dtype == "O") & (len(x.unique()) <= 10)) else x, axis=0).isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64